In [1]:
from src.io_utils import load_data, coerce_numeric

df = coerce_numeric(load_data("data/library_loans.csv"))

# Removes rows with missing or non-positive loan days
df = df.dropna(subset=["loan_days"]).copy()
df = df[df["loan_days"] > 0].copy()

df["month"] = df["checkout_date"].dt.to_period("M").dt.to_timestamp()
df["week"] = df["checkout_date"].dt.to_period("W").dt.start_time

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 124 entries, 0 to 127
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   loan_id           124 non-null    object        
 1   checkout_date     124 non-null    datetime64[ns]
 2   branch            121 non-null    object        
 3   genre             118 non-null    object        
 4   item_type         123 non-null    object        
 5   patron_age_group  123 non-null    object        
 6   loan_days         124 non-null    int64         
 7   returned_date     108 non-null    datetime64[ns]
 8   overdue_days      114 non-null    float64       
 9   fine_amount       113 non-null    float64       
 10  month             124 non-null    datetime64[ns]
 11  week              124 non-null    datetime64[ns]
dtypes: datetime64[ns](4), float64(2), int64(1), object(5)
memory usage: 12.6+ KB


,loan_id,checkout_date,branch,genre,item_type,patron_age_group,loan_days,returned_date,overdue_days,fine_amount,month,week
0,L0005,2025-08-05,Väster,Romantik,E-bok,Vuxen,18,2025-08-23,0.0,NaN,2025-08-01,2025-08-04
1,L0041,2025-08-30,Öster,Biografi,Tidskrift,Vuxen,23,2025-09-22,1.0,2.935475,2025-08-01,2025-08-25
2,L0091,2025-03-19,Väster,Biografi,E-bok,Vuxen,23,2025-04-11,2.0,13.479296,2025-03-01,2025-03-17
3,L0083,2025-06-12,Öster,Fakta,Tidskrift,Vuxen,21,2025-07-03,0.0,0.000000,2025-06-01,2025-06-09
4,L0043,2025-08-09,Söder,Biografi,Ljudbok,Vuxen,23,2025-09-01,1.0,6.428793,2025-08-01,2025-08-04


In [2]:
from src import metrics as M

print("Total loans:", M.total_loans(df))
print(f"Average loan days: {M.average_loan_days(df):.1f}")
print(f"Overdue rate: {100.0 * M.overdue_rate(df):.1f}%")

Total loans: 116
Average loan days: 21.3
Overdue rate: 47.6%


In [5]:
by_genre = M.loans_by_genre(df)
by_branch = M.loans_by_branch(df)
by_genre, by_branch

(      genre  loans
 0  Biografi     22
 1     Fakta     22
 2   Deckare     20
 3    Sci-Fi     17
 4  Romantik     17
 5      Barn     16
 6       NaN      6,
     branch  loans
 0    Öster     32
 1   Väster     28
 2     Norr     21
 3  Centrum     20
 4    Söder     13
 5      NaN      3)